In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ""

In [2]:
from llava.mm_utils import get_model_name_from_path, tokenizer_image_token
from llava.model.builder import load_pretrained_model
from llava.constants import IMAGE_TOKEN_INDEX
from llava.eval.run_llava import eval_model
from PIL import Image
import torch

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


[2024-07-15 13:23:59,971] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
model_path = "../models/llava-v1.6-vicuna-7b-VD/checkpoint-12000"
device = 'cpu'

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=get_model_name_from_path(model_path)
)

/home/et/miniconda3/envs/as-visual_description/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.
/home/et/miniconda3/envs/as-visual_description/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/et/miniconda3/envs/as-visual_description/lib/python3.11/site-packages/torch/nn/modules/mo

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [5]:
prompt = """USER: You are an AI model designed to help visually impaired people. Your task is to provide a comprehensive description of the image, locating important objects to guide disabled people through the scene.\nBelow is an office scene. Provide a descriptive paragraph using a human-like description. Do not add any comment or guess. Remain factual and avoid unnecessary embellishments, keep it simple.\n<image> ASSISTANT:"""
image_file = '../data/image_examples/office.jpeg'

raw_image = Image.open(image_file).convert('RGB')
image_tensor = image_processor(raw_image, return_tensors='pt')['pixel_values'].to(device).to(torch.float16)
input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(device)

with torch.inference_mode():
    output_ids = model.generate(
                        input_ids,
                        images=image_tensor,
                        do_sample=False,
                        #temperature=0.2,
                        max_new_tokens=500,
                        )

outputs = tokenizer.decode(output_ids[0,:]).strip()
print(outputs)

RuntimeError: "slow_conv2d_cpu" not implemented for 'Half'